In [1]:
from pynwb import NWBFile
from neo import *
import pynwb
from pynwb import *
import quantities as pq
from quantities import s, ms,kHz, Hz, uV
import numpy as np
from datetime import datetime
import nwbinspector

## Define Neo structure and NWB annotations

In [2]:
subject_annotations = {
                        "nwb:subject_id": "012",
                        "nwb:age": "P90D",
                        "nwb:description": "mouse 5",
                        "nwb:species": "Mus musculus",
                        "nwb:sex": "M",
        }

annotations = {
                "session_start_time": datetime.now(),
                "subject": subject_annotations,
              }
# Define Neo blocks

bl0 = Block(name='First block', **annotations)
#print("bl0.annotations = ", bl0.annotations)
bl1 = Block(name='Second block', **annotations)
original_blocks = [bl0, bl1]

# Neo structure
for blk in original_blocks:
    for ind in range(3):  # number of Segments
        seg = Segment(index=ind)
        seg.block = blk
        blk.segments.append(seg)

    for seg in blk.segments:

        original_block = Block(name="experiment", 
                               session_start_time=datetime.now(), 
                               experimenter="Elodie", 
                               experiment_description="Experiment description",
                               institution="CNRS",
                               subject=subject_annotations,
                              )
#        print("original_block.annotations = ", original_block.annotations)
        segment = Segment(name="session 1")
        original_block.segments.append(segment)
        segment.block = original_block

        # NWB annotations
        electrode_annotations = {
            "name": "electrode #1",
            "description": "intracellular electrode",
            "device": {"name": "electrode #1"},}

        sweep_number_annotations = {
            "name": ("pynwb.icephys", "SweepTable"),
            "description": "Description of the SweepTable",
            "id": 1.0,
            "columns":1,
            "colnames":1,}

        stimulus_annotations = {
            "nwb_group": "stimulus",
            "nwb_neurodata_type": ("pynwb.icephys", "CurrentClampStimulusSeries"),
            "nwb_electrode": electrode_annotations,
            "nwb_sweep_number": sweep_number_annotations,
            "nwb:gain": 1.0,}

        response_annotations = {
            "nwb_group": "acquisition",
            "nwb_neurodata_type": ("pynwb.icephys", "CurrentClampSeries"),
            "nwb_electrode": electrode_annotations,
            "nwb_sweep_number": sweep_number_annotations,
            "nwb:gain": 1.0,
            "nwb:bias_current": 1e-12,
            "nwb:bridge_balance": 70e6,
            "nwb:capacitance_compensation": 1e-12,}

        # Neo objects
        stimulus = AnalogSignal(np.random.randn(100, 1) * pq.nA,
                                sampling_rate=5 * pq.kHz,
                                t_start=50 * pq.ms,
                                name="stimulus",
                                **stimulus_annotations)
        response = AnalogSignal(np.random.randn(100, 1) * pq.mV,
                                sampling_rate=5 * pq.kHz,
                                t_start=50 * pq.ms,
                                name="response",
                                **response_annotations)
        
        segment.analogsignals = [stimulus, response]
        stimulus.segment = response.segment = segment

## Write a NWB file using Neo NWBIO

In [3]:
test_file_name = "test_NWB_Inspector.nwb"
iow = NWBIO(filename=test_file_name, mode='w')
iow.write_all_blocks([original_block])
print("iow = ", iow)

/Users/legouee/miniconda3/envs/Brainhack/lib/python3.8/site-packages/pynwb/file.py:1037: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")
/Users/legouee/miniconda3/envs/Brainhack/lib/python3.8/site-packages/pynwb/icephys.py:21: UserWarning: Unit 'V' for CurrentClampStimulusSeries 'experiment : segment0 stimulus 0' is ignored and will be set to 'amperes' as per NWB 2.1.0.
  warnings.warn(
/Users/legouee/miniconda3/envs/Brainhack/lib/python3.8/site-packages/pynwb/icephys.py:21: UserWarning: Unit 'V' for CurrentClampSeries 'experiment : segment0 response 1' is ignored and will be set to 'volts' as per NWB 2.1.0.
  warnings.warn(


iow =  <neo.io.nwbio.NWBIO object at 0x11263c340>


## Read this file using Neo NWBIO

In [4]:
ior = NWBIO(filename=test_file_name, mode='r')
retrieved_block = ior.read_all_blocks()
print("retrieved_block = ", retrieved_block)

retrieved_block =  [<neo.core.block.Block object at 0x12a1c0a00>]


## Validating NWB file using PyNWB

In [5]:
!python -m pynwb.validate test_NWB_Inspector.nwb

Validating test_NWB_Inspector.nwb against cached namespace information using namespace 'core'.
 - no errors found.


## NWB Inspector
### Inspect NWB files for compliance with NWB Best Practices. 

### Quick check

In [6]:
!nwbinspector /Users/legouee/NWBwork/my_notebook/test_NWB_Inspector.nwb

## Common flags

### Save the report for future reference

In [7]:
#!nwbinspector /Users/legouee/NWBwork/my_notebook/test_NWB_Inspector.nwb -o --log-file /Users/legouee/NWBwork/my_notebook/nwbinspector_report.txt
!nwbinspector /Users/legouee/NWBwork/my_notebook/test_NWB_Inspector.nwb -o --json-file-path /Users/legouee/NWBwork/my_notebook/nwbinspector_report.txt

## Inspect a single NWBFile

In [8]:
from nwbinspector import inspect_nwb

results = list(inspect_nwb(nwbfile_path="test_NWB_Inspector.nwb"))
print("results = ", results)

results =  []


## Yielding and Iterating

In [9]:
results_generator = inspect_nwb(nwbfile_path="test_NWB_Inspector.nwb")
print("results_generator = ", results_generator)

results_generator =  <generator object inspect_nwb at 0x129f3c350>


In [10]:
#first_message = next(results_generator)
#print("first_message = ", first_message)

In [11]:
results_generator = inspect_nwb(nwbfile_path="test_NWB_Inspector.nwb")

for message in results_generator:
    print(message.message)
    print("  ")
    print(message)
    print("   ")
    print("message.importance=", message.importance)
    print("message.importance.__dict__=", message.importance.__dict__)
    
    print("message.severity=", message.severity)
    print("message.severity.__dict__=", message.severity.__dict__)
    print("message.severity=", message.severity)
    
    
    if message.importance._name_=="BEST_PRACTICE_SUGGESTION":
        print("ok")
    if message.severity._name_=="LOW":
        print("OKok")
    print("----")